In [199]:
#-----------------------------------------------------------------------------
# Cleaning of DRIAMS-B id file
#
# May 2018 C. Weis
#-----------------------------------------------------------------------------

import os
import csv
import numpy as np
import pandas as pd

from collections                import defaultdict as ddict

%load_ext autoreload
%autoreload 2

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [200]:
save_IDRES_table = True

Goal: Convert Validation dataset csv files into established format of Maldi project

In [201]:
USB_antibiotic_list = ['Ciprofloxacin', 'Cefepim', 'Meropenem', 'Piperacillin...Tazobactam', 'Cotrimoxazol', 
                       'Ceftazidim', 'Levofloxacin', 'Colistin', 'Tobramycin', 'Ceftriaxon', 'Imipenem', 'Amikacin', 
                       'Tigecyclin', 'Clindamycin', 'Amoxicillin...Clavulansaeure', 'Amoxicillin', 'Posaconazol', 
                       'Itraconazol', 'Voriconazol', 'Caspofungin', 'Amphotericin.B', 'Penicillin', 'Vancomycin', 
                       'Ertapenem', 'Metronidazol', 'Moxifloxacin', 'Rifampicin', 'Erythromycin', 'Fluconazol', 
                       'Anidulafungin', 'X5.Fluorocytosin', 'Micafungin', 'Ampicillin...Amoxicillin', 'Norfloxacin', 
                       'Fosfomycin.Trometamol', 'Cefpodoxim', 'Chloramphenicol', 'Aminoglykoside', 'Chinolone', 
                       'Daptomycin', 'Teicoplanin', 'Linezolid', 'Gentamicin', 'Gentamicin.High.level', 
                       'Nitrofurantoin', 'Cefuroxim', 'Meropenem.ohne.Meningitis', 'Meropenem.bei.Meningitis', 
                       'Ceftazidim.1', 'Fosfomycin', 'Aztreonam', 'Cefazolin', 'Tetracyclin', 'Fusidinsaeure', 
                       'Oxacillin', 'Clarithromycin', 'Isoniazid.0.1.mg.l', 'Streptomycin.1.0.mg.l', 
                       'Rifampicin.1.0.mg.l', 'Ethambutol.5.0.mg.l', 'Pyrazinamid.100.0.mg.l', 'Azithromycin', 
                       'Cefixim', 'Doxycyclin', 'Mupirocin', 'Vancomycin.GRD', 'Teicoplanin.GRD', 'Cefoxitin.Screen', 
                       'Ceftarolin', 'Ticarcillin...Clavulansaeure', 'Penicillin.bei.Endokarditis', 
                       'Penicillin.ohne.Meningitis', 'Penicillin.ohne.Endokarditis', 'Dummy', 
                       'Penicillin.bei.Pneumonie', 'Penicillin.bei.anderen.Infekten', 'Penicillin.bei.Meningitis', 
                       'Meropenem.bei.Pneumonie', 'Cefepim.1', 'Minocyclin', 'Cefuroxim.Axetil', 
                       'Amoxicillin...Clavulansaeure.bei.unkompliziertem.HWI', 'Ceftazidim.Avibactam', 
                       'Ceftolozan...Tazobactam', 'Ampicillin...Sulbactam', 'Ceftobiprol', 'Bacitracin', 
                       'Isoniazid.0.4.mg.l', 'Streptomycin.High.level', 'Isavuconazol']

In [202]:
INPUT_file = '/links/groups/borgwardt/Projects/maldi_tof_diagnostics/amr_maldi_ml/codeAC/KSBL/KSBL_res_report.csv'
OUTPUT_file = '/links/groups/borgwardt/Data/DRIAMS/DRIAMS-B/id/2018/2018_clean.csv'

In [204]:
# TODO change name matching to general script, not from AB-matching.csv
def get_antibiotics_name_matching(match_from='LIESTAL', match_to='USB'):
    assert match_from in ['LIESTAL','USB','Viollier','Aarau','Madrid']
    assert match_to in ['LIESTAL','USB','Viollier','Aarau','Madrid']
    csvname = '/links/groups/borgwardt/Projects/maldi_tof_diagnostics/amr_maldi_ml/MaldiML/MaldiML/files/AB-matching.csv'

    with open(csvname,'r', encoding='mac_roman') as f:
        ff = csv.reader(f, delimiter=',', dialect=csv.excel)
        list_antibiotics = []

        d_naming = ddict(list)

        for j, row in enumerate(ff):

            if j==0:
                inidx = row.index(match_from)
                outidx = row.index(match_to)
            else:
                inname = row[inidx]
                outname = row[outidx]
                if inname == '':
                    continue
                else:
                    d_naming[inname] = outname
    return d_naming

In [205]:
d_matching = get_antibiotics_name_matching(match_from='LIESTAL', match_to='USB')
d_matching['Cefuroxim oral'] = ''

In [206]:
d_matching

defaultdict(list,
            {'Amikacin': 'Amikacin',
             'Amoxicillin/Clavulansaeure': 'Amoxicillin...Clavulansaeure',
             'Ampicillin': '',
             'Benzylpenicillin': '',
             'Benzylpenicillin andere': '',
             'Benzylpenicillin bei Meningitis': '',
             'Benzylpenicillin bei Pneumonie': '',
             'Cefepime': 'Cefepim',
             'Cefoxitin Screening Staph': 'Cefoxitin.Screen',
             'Cefoxitin': '',
             'Ceftazidim': 'Ceftazidim',
             'Ceftriaxon': 'Ceftriaxon',
             'Cefuroxim iv': 'Cefuroxim',
             'Cefuroxim oral': '',
             'Ciprofloxacin': 'Ciprofloxacin',
             'Clarithromycin': 'Clarithromycin',
             'Clindamycin': 'Clindamycin',
             'Clindamycin ind.': '',
             'Trimethoprim/Sulfamethoxazol': 'Cotrimoxazol',
             'Daptomycin': 'Daptomycin',
             'ESBL': '',
             'Ertapenem': 'Ertapenem',
             'Erythromycin

# Kantonsspital Basel-Land KSBL

Every antibiotic resistance is its only line, so one spectra has multiple lines in the file.

In [207]:
# make antibiotics inventory first

with open(INPUT_file,'r') as f:
    ff = csv.reader(f, delimiter=';', dialect=csv.excel)
    list_antibiotics = []
    
    for j, row in enumerate(ff):
        if j==0:
            header = list(row)
            idx_antibiotic = header.index('Antibiotic')
        else:
            if row[idx_antibiotic] in d_matching.keys():
                list_antibiotics.append(d_matching[row[idx_antibiotic]])

list_antibiotics_KSBL = np.unique(list_antibiotics)

union_USB_KSBL = set(USB_antibiotic_list) & set(list_antibiotics_KSBL)

In [208]:
# make list of BrukerIDs that are unique w.r.t. spectra scores, and strain from lab results
#
# this list is used by convert_csv function to subset the spectra that will be considered
#
with open(INPUT_file,'r') as f:
    ff = csv.reader(f, delimiter=';', dialect=csv.excel)
    
    d = ddict(list)
    for j, row in enumerate(ff):
        if j==0:
            header = list(row)
            idx_Tagnr = header.index('Auftrag')
            idx_Bruker = header.index('Bruker')
            idx_species = header.index('Keim')
            idx_score1 = header.index('Score1')
        else:
            d[row[idx_Bruker]].append(row[idx_Tagnr])
            d[row[idx_Bruker]].append(row[idx_species])
            d[row[idx_Bruker]].append(row[idx_score1])

list_sampleID_unique = [key for key in d.keys() if len(np.unique(d[key]))==3]

In [209]:
# Read in csv file
def convert_csv(csvname, list_sampleIDs, list_antibiotics, d_ab_match, id_colname='Bruker', ab_colname='Antibiotic', rsi_colname='Resultat', species_colname='Organism_best_match' , genus_colname='GENUS'):
    with open(csvname,'r') as f:
        ff = csv.reader(f, delimiter=';', dialect=csv.excel)

        currentBrukerID = ''
        completeDataset_dict = ddict(list)

        # initialize all antibiotics
        for k in ['species', 'genus', 'code']:
            completeDataset_dict[k].append('')
        for ab in list_antibiotics:
            completeDataset_dict[ab].append('')

        for j, row in enumerate(ff):

            if j==0:
                header = list(row)
                
                idx_Bruker = header.index(id_colname)
                idx_antibiotic = header.index(ab_colname)
                idx_RSI = header.index(rsi_colname)

                idx_species = header.index(species_colname)
                idx_genus = header.index(genus_colname)

            else:
                BrukerID = list(row)[idx_Bruker]

                if BrukerID not in list_sampleIDs:
                    continue

                # if sample exists, continue appending
                elif BrukerID == currentBrukerID:
                    if row[idx_antibiotic] in d_ab_match.keys():
                        if d_ab_match[row[idx_antibiotic]] in list_antibiotics:
                            completeDataset_dict[d_ab_match[row[idx_antibiotic]]].append(row[idx_RSI])

                    assert currentSpecies == row[idx_species]
                    assert currentGenus == row[idx_genus]

                # start new sample
                else:
                    if j>1:
                        # save sample before starting with a new one
                        completeDataset_dict['code'].append(currentBrukerID)
                        completeDataset_dict['genus'].append(currentGenus)
                        completeDataset_dict['species'].append(currentSpecies)

                        for ab in list_antibiotics:
                            if len(completeDataset_dict[ab]) < len(completeDataset_dict['species']):
                                completeDataset_dict[ab].append('nan')
                        
                        # each value (e.g. each column in our dataframe) has the same length
                        x = [len(value) for key, value in completeDataset_dict.items()]
                        assert len(np.unique(x)) == 1

                    # start new sample
                    currentBrukerID = BrukerID
                    currentSpecies = row[idx_species]
                    currentGenus = row[idx_genus]

                    # start appending antibiotics resistances
                    if row[idx_antibiotic] in d_ab_match.keys():
                        if d_ab_match[row[idx_antibiotic]] in list_antibiotics:
                            completeDataset_dict[d_ab_match[row[idx_antibiotic]]].append(row[idx_RSI])


    # finish last sample
    completeDataset_dict['code'].append(currentBrukerID)
    completeDataset_dict['genus'].append(currentGenus)
    completeDataset_dict['species'].append(currentSpecies)

    for ab in list_antibiotics:
        if len(completeDataset_dict[ab]) < len(completeDataset_dict['species']):
            completeDataset_dict[ab].append('nan')

    # each value (e.g. each column in our dataframe) has the same length
    x = [len(value) for key, value in completeDataset_dict.items()]
    assert len(np.unique(x)) == 1

    KSBL_Dataset = pd.DataFrame.from_dict(completeDataset_dict)
    
    # clean up dataframe
    KSBL_Dataset = KSBL_Dataset.drop(columns=['genus'])
    KSBL_Dataset = KSBL_Dataset.replace({'RES': 'R', 'INT': 'I','negativ': '0', 'positiv': '1'})
    #remove initialization line
    KSBL_Dataset.drop(KSBL_Dataset.index[0], inplace=True) 
    
    # assert only entries in ['0', '1', 'I', 'R', 'S', 'nan']
    unique_vals = np.unique(KSBL_Dataset[list(KSBL_Dataset.drop(columns=['code','species']).columns)].values)
    assert np.all([val in ['0', '1', 'I', 'R', 'S', 'nan'] for val in unique_vals])
    
    return KSBL_Dataset


In [210]:
# save table to file
KSBL_Dataset = convert_csv(INPUT_file, list_sampleID_unique, union_USB_KSBL, d_matching, species_colname='Organism_best_match')

if save_IDRES_table:
    KSBL_Dataset.to_csv(OUTPUT_file, sep=',', encoding='utf-8', index=False)
else:
    print(KSBL_Dataset.head())

['0' '1' 'I' 'R' 'S' 'nan']
